In [1]:
# general tools
import os
import re
import sys
import netCDF4 as nc
from glob import glob

import numpy as np
from datetime import datetime, timedelta

In [2]:
lead = 4

In [7]:
model_tag = 'peak' #args['model_tag']
filepath_vec = "/glade/work/ksha/NCAR/"

In [8]:
path_name = '/glade/campaign/cisl/aiml/ksha/NCAR_batch_v4_temp/'

In [9]:
filename_valid = sorted(glob("{}TEST*lead{}.npy".format(path_name, lead)))

In [10]:
len(filename_valid)

375220

In [11]:
#filename_valid
valid_lead = np.load('{}TEST_v4_vec_lead{}_{}.npy'.format(filepath_vec, lead, model_tag), allow_pickle=True)[()]

In [12]:
VALID_lead_y = valid_lead['y_true']

In [13]:
IND_VALID_lead = np.load('/glade/work/ksha/NCAR/IND_TEST_lead_v4.npy', allow_pickle=True)[()]
VALID_ind = IND_VALID_lead['lead{}'.format(lead)]

In [14]:
L = len(VALID_ind)

filename_valid_pick = []

VALID_Y = np.zeros(L)

for i in range(L):
    
    ind_lead = int(VALID_ind[i])
    filename_valid_pick.append(filename_valid[ind_lead])

    if 'pos' in filename_valid[ind_lead]:
        if VALID_lead_y[ind_lead] == 1.0:
            VALID_Y[i] = 1.0
        else:
            egwrshat

In [15]:
def id_extract(filenames):
    
    indx_out = []
    indy_out = []
    lead_out = []
    day_out = []
    
    for i, name in enumerate(filenames):        
        nums = re.findall(r'\d+', name)
        lead = int(nums[-1])
        indy = int(nums[-2])
        indx = int(nums[-3])
        day = int(nums[-4])
                
        indx_out.append(indx)
        indy_out.append(indy)
        lead_out.append(lead)
        day_out.append(day)
        
    return np.array(indx_out), np.array(indy_out), np.array(lead_out), np.array(day_out)

In [16]:
indx, indy, _, day = id_extract(filename_valid_pick)

In [17]:
with nc.Dataset(filepath_vec+'climo_severe_2hr_all.nc') as ncio:
    climo = ncio['climo'][...]

In [18]:
base_v3_s = datetime(2018, 7, 15)
base_v3_e = datetime(2020, 12, 2)

base_v4_s = datetime(2020, 12, 3)
base_v4_e = datetime(2022, 7, 15)

base_ref = datetime(2010, 1, 1)

date_list_v3 = [base_v3_s + timedelta(days=day) for day in range(365+365+142)]
date_list_v4 = [base_v4_s + timedelta(days=day) for day in range(365+180)]

In [22]:

probs = []
for i in range(L):
    
    date_ = date_list_v4[day[i]]
    year_ = date_.year
    ind_day = int((date_ - datetime(year_, 1, 1)).days)
    
    probs.append(climo[0, ind_day, lead-1, indx[i], indy[i]])

In [23]:
climos = np.array(probs)

In [24]:
np.mean((VALID_Y - climos)**2)

0.0035614021871754834

In [25]:
np.mean(VALID_Y)

0.003583721725745072